In [1]:
#dependencies 
#had to 
import pandas as pd 
from requests import Request, Session
from requests.exceptions import ConnectionError, Timeout, TooManyRedirects
import json
import config 
from coincallandwritepythonfile import refined_coin_list

In [2]:
refined_coin_list

,id,name,symbol,rank,File_Name,injest_datetime
1,1,Bitcoin,BTC,1,Coin_Ranking_Dim,04/20/2022 15:45:17
2,1027,Ethereum,ETH,2,Coin_Ranking_Dim,04/20/2022 15:45:17
3,825,Tether,USDT,3,Coin_Ranking_Dim,04/20/2022 15:45:17
4,1839,BNB,BNB,4,Coin_Ranking_Dim,04/20/2022 15:45:17
5,3408,USD Coin,USDC,5,Coin_Ranking_Dim,04/20/2022 15:45:17
...,...,...,...,...,...,...
97,2586,Synthetix,SNX,97,Coin_Ranking_Dim,04/20/2022 15:45:17
98,4846,Kava,KAVA,98,Coin_Ranking_Dim,04/20/2022 15:45:17
99,9444,Kyber Network Crystal v2,KNC,99,Coin_Ranking_Dim,04/20/2022 15:45:17
100,2777,IoTeX,IOTX,100,Coin_Ranking_Dim,04/20/2022 15:45:17


In [3]:
#https://coinmarketcap.com/api/documentation/v1/#operation/getV2CryptocurrencyQuotesLatest
#now need to reference the coin ids and pull in data from the api accordinly 

config.coin_api_key

url = 'https://pro-api.coinmarketcap.com/v2/cryptocurrency/quotes/latest'
parameters = {
    'id': [1]
    }
headers = {
'Accepts': 'application/json',
'X-CMC_PRO_API_KEY': config.coin_api_key
}

session = Session()
session.headers.update(headers)
response = session.get(url, params=parameters)
data = json.loads(response.text)


In [4]:
#navigate json
data['data']
#read to DF to see what we have
cmc_df = pd.DataFrame.from_dict(data['data'], orient='index')
#need to explode this, then re join 
cmc_df['quote'][0]

,id,name,symbol,slug,num_market_pairs,date_added,tags,max_supply,circulating_supply,total_supply,is_active,platform,cmc_rank,is_fiat,self_reported_circulating_supply,self_reported_market_cap,last_updated,quote
1,1,Bitcoin,BTC,bitcoin,9340,2013-04-28T00:00:00.000Z,"[{'slug': 'mineable', 'name': 'Mineable', 'cat...",21000000,19017287,19017287,1,None,1,0,None,None,2022-04-20T22:44:00.000Z,"{'USD': {'price': 41546.639963689806, 'volume_..."


In [53]:
#expand nest columns
expanded_df = pd.DataFrame.from_dict(cmc_df['quote'][0], orient= 'index')
#showing USD as an index column, going to drop
expanded_df.reset_index(drop = True, inplace= True)
#can double check shape
expanded_df.shape
trial = cmc_df.drop(['quote','tags'], axis= 1)
#had to reset index or set it manually to match the expanded DF 
trial.set_index(pd.Index([0]), inplace= True)
#trial.reset_index(drop = True, inplace= True)

coin_price_df = pd.concat([trial,expanded_df], axis=1 )

#going to try to append this to another dataframe
df = pd.DataFrame()

df = df.append(coin_price_df)
df.head()

,id,name,symbol,slug,num_market_pairs,date_added,max_supply,circulating_supply,total_supply,is_active,...,percent_change_1h,percent_change_24h,percent_change_7d,percent_change_30d,percent_change_60d,percent_change_90d,market_cap,market_cap_dominance,fully_diluted_market_cap,last_updated
0,1,Bitcoin,BTC,bitcoin,9340,2013-04-28T00:00:00.000Z,21000000,19017287,19017287,1,...,0.208146,0.288073,0.536021,0.829391,3.857871,0.565335,7.901044e+11,41.1164,8.724794e+11,2022-04-20T22:44:00.000Z


In [52]:
df

""


In [48]:
coin_price_df

,id,name,symbol,slug,num_market_pairs,date_added,max_supply,circulating_supply,total_supply,is_active,...,percent_change_1h,percent_change_24h,percent_change_7d,percent_change_30d,percent_change_60d,percent_change_90d,market_cap,market_cap_dominance,fully_diluted_market_cap,last_updated
0,1,Bitcoin,BTC,bitcoin,9340,2013-04-28T00:00:00.000Z,21000000,19017287,19017287,1,...,0.208146,0.288073,0.536021,0.829391,3.857871,0.565335,7.901044e+11,41.1164,8.724794e+11,2022-04-20T22:44:00.000Z


In [25]:
trial = cmc_df.drop(['quote','tags'], axis= 1)
trial.shape

(1, 16)

In [27]:
coin_price_df

,id,name,symbol,slug,num_market_pairs,date_added,max_supply,circulating_supply,total_supply,is_active,...,percent_change_1h,percent_change_24h,percent_change_7d,percent_change_30d,percent_change_60d,percent_change_90d,market_cap,market_cap_dominance,fully_diluted_market_cap,last_updated
1,1.0,Bitcoin,BTC,bitcoin,9340.0,2013-04-28T00:00:00.000Z,21000000.0,19017287.0,19017287.0,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.208146,0.288073,0.536021,0.829391,3.857871,0.565335,7.901044e+11,41.1164,8.724794e+11,2022-04-20T22:44:00.000Z


In [26]:
768620549906.7964/ 40426.70598566949

19012693.0

In [14]:
trial.index

Index(['USD'], dtype='object')